In [25]:
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 
  
class TwitterClient(object): 
    ''' 
    Generic Twitter Class for sentiment analysis. 
    '''
    def __init__(self): 
        ''' 
        Class constructor or initialization method. 
        '''
        # keys and tokens from the Twitter Dev Console 
        consumer_key = 'pBB1FkSwZCcBqtr6LScpZt8fy'
        consumer_secret = '1iMBLqIa2C2FBosbK4AHP4h782WG1EJrxPBYGs9xOiuPikkY41'
        access_token = '1194438905224945664-LP1Xu8nHNLH4YgOwHrZpoa2KwHaQzA'
        access_token_secret = 'GQQP6ltkhQVvj1vxRYPFWDdtN0CSYRXGvM70Go9WdwjYj'
  
        # attempt authentication 
        try: 
            # create OAuthHandler object 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            self.auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed") 
  
    def clean_tweet(self, tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 
  
    def get_tweet_sentiment(self, tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(self.clean_tweet(tweet)) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'
  
    def get_tweets(self, query, count = 10): 
        ''' 
        Main function to fetch tweets and parse them. 
        '''
        # empty list to store parsed tweets 
        tweets = [] 
  
        try: 
            # call twitter api to fetch tweets 
            fetched_tweets = self.api.search(q = query, count = count) 
  
            # parsing tweets one by one 
            for tweet in fetched_tweets: 
                # empty dictionary to store required params of a tweet 
                parsed_tweet = {} 
  
                # saving text of tweet 
                parsed_tweet['text'] = tweet.text 
                # saving sentiment of tweet 
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 
  
                # appending parsed tweet to tweets list 
                if tweet.retweet_count > 0: 
                    # if tweet has retweets, ensure that it is appended only once 
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet) 
  
            # return parsed tweets 
            return tweets 
  
        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e)) 
  
def main(): 
    # creating object of TwitterClient Class 
    api = TwitterClient() 
    # calling function to get tweets 
    tweets = api.get_tweets(query = 'Free Hong Kong', count = 200) 
  
    # picking positive tweets from tweets 
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
    # percentage of positive tweets 
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
    # picking negative tweets from tweets 
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
    # percentage of negative tweets 
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
#     # percentage of neutral tweets 
#     print("Neutral tweets percentage: {} %".format(100*(len(tweets) - len(ntweets) - len(ptweets)/len(tweets)) 
  
    # printing first 5 positive tweets 
    print("\n\nPositive tweets:") 
    for tweet in ptweets[:10]: 
        print(tweet['text']) 
  
    # printing first 5 negative tweets 
    print("\n\nNegative tweets:") 
    for tweet in ntweets[:10]: 
        print(tweet['text']) 
  
if __name__ == "__main__": 
    # calling main function 
    main()


Positive tweets percentage: 80.76923076923077 %
Negative tweets percentage: 11.538461538461538 %


Positive tweets:
RT @CleanbrexitBill: We see all the troubles in Hong Kong? Why NOTHING on the Paris troubles, must be an Eu directive? Free speech &amp; democr…
Everyone Talking 'Free Hong Kong' Until They Meet This Motherfucker https://t.co/dPaSxBakwa
RT @shiroihamusan: @SenRickScott Thank you Senator Scott for the great support to HKers
CCP is a threat to the whole world, hope all people…
@Reuters Free Hong Kong !
RT @zherring: I just finished. Strongly encourage reading this. 

If we're allies of people's self-determinism, "Free the Uyghurs" is as ur…
RT @dcexaminer: Some "yellow vest" protesters in Paris marked the 1 yr anniversary of their organization by leaving pro-democracy messages…
RT @clyde0313: #StandWithHongKong 
If U feel free, please check what’s happening In Hong Kong https://t.co/yt5jpb1FAO
@lampat_lal @Reuters Hong Kong is part of China and is free like rest of the coun